In [1]:
#マルチターンデータセットを統合する

In [2]:
import glob
import json
import re
jsonl_path_list=glob.glob("data_multi_*/*.jsonl")

In [3]:

records=[]
for jsonl_path in jsonl_path_list:
    with open(jsonl_path,"r") as f:
        for line in f:
            records.append(json.loads(line))

In [4]:

#きれいにする
remove_words=[
"User:",
"Assistant:",
"ユーザー：",
"アシスタント：",
"ユーザー:",
"アシスタント:",
]
def clean_chat(txt):
    for word in remove_words:
        if txt.startswith(word):
            txt=txt[len(word):]
    txt=txt.strip()
    if txt[0]=="「" and txt[-1]=="」":
        txt=txt[1:-1]


    return txt

In [5]:
txt="「アートの世界において、巨匠と呼ばれる人物は誰がありますか？」"
clean_chat(txt)

'アートの世界において、巨匠と呼ばれる人物は誰がありますか？'

In [6]:
def parse_full_auto_dialogue(txt):
    if "### 指示1:" in txt and "### 応答1:" and "### 指示2:" in txt and "### 応答2:":
        # 正規表現で「指示」と「応答」を抽出
        pattern = r"### 指示(\d+):(.*?)\n### 応答\1:(.*?)(?=\n### 指示|$)"
        matches = re.findall(pattern, txt, re.DOTALL)

        # 辞書に変換
        dialogue_dict = {}
        add_flag=True
        for match in matches:
            idx = int(match[0])
            if idx>=3:
                continue
            user_text = match[1].strip()
            assistant_text = match[2].strip()
            if len(user_text)<3:
                add_flag=False
                break
            if assistant_text=="":
                add_flag=False
                break

            if user_text.find("以下の情報を元に、UserとAssistantのやりとりを")>=0:
                add_flag=False
                break

            dialogue_dict[f"q{idx}"] = clean_chat(user_text)
            dialogue_dict[f"a{idx}"] = clean_chat(assistant_text)

        #最後にチェック
        if "q1" in dialogue_dict and "q2" in dialogue_dict and "a1" in dialogue_dict and "a2" in dialogue_dict:
            return dialogue_dict,add_flag
    
    return {},False

In [7]:
record=records[1]
dialogues=[]


for record in records:

    #自動生成のdialogue
    if "autogen_text" in record:
        txt=record["autogen_text"].strip()

    #質問を与える場合
    elif "question" in record:
        txt=record["response"].strip()
    else:
        raise ValueError(record)

    dialogue_dict,add_flag=parse_full_auto_dialogue(txt)
    dialogue_dict["database"]=record["database"]
    if add_flag:
        dialogues.append(dialogue_dict)

In [8]:
record

{'question': '最近の忙しさの中で、自分が好きなフィットネスクラスに参加することが出来なくなってきました。仕事や家事、子育てのために毎日の時間はいっぱいで、スポーツを続けることが難しくなりました。しかし、健康は大切です。私たちはどのようにしてフィットネスクラスの習慣を取り組むべきでしょうか？',
 'database': 'minnnade',
 'response': ' ### 指示1:最近の忙しさの中で、自分が好きなフィットネスクラスに参加することが出来なくなってきました。仕事や家事、子育てのために毎日の時間はいっぱいで、スポーツを続けることが難しくなりました。しかし、健康は大切です。私たちはどのようにしてフィットネスクラスの習慣を取り組むべきでしょうか？\n### 応答1:わかりました。時間が限られている状態でフィットネスクラスに参加するのは難しいですね。しかし、健康のために継続することは大切です。最初に考えられるのは、自分の時間表を調整してフィットネスクラスに参加する時間を確保することです。\n### 指示2:具体的にどのように時間表を調整すればいいのですか？\n### 応答2:具体的な方法は、毎日のスケジュールを確認して、フィットネスクラスに参加する時間帯を決定することが重要です。例えば、仕事や家事、子育てのタイミングを調整して、毎週3回はフィットネスクラスに参加する時間帯を確保します。また、スケジュールが変更された場合は、予め代替日を決定しておくのも良い方法です。こうして時間表を調整することで、フィットネスクラスに参加する習慣を取り込むことが出来ます。'}

In [9]:
import datasets
import pandas as pd
from huggingface_hub import HfApi, logging
df=pd.DataFrame(dialogues)
df=df.reindex()
parquet_path="data_multi_from_tsubame/1.parquet"
df.to_parquet(parquet_path)

jsonl_path="data/multi.jsonl"
with open(jsonl_path,"w") as f:
    for dialogue in dialogues:
        f.write(json.dumps(dialogue,ensure_ascii=False)+"\n")

hf = HfApi()
logging.set_verbosity_debug()
hf.upload_file(#path_or_fileobj=parquet_path,
                path_or_fileobj=jsonl_path,
                #path_in_repo=f"1.parquet",
                path_in_repo=f"1.jsonl",
                repo_id="kanhatakeyama/AutoMultiTurnByMixtral8x22b", repo_type="dataset")

/home/user/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
About to commit to the hub: 1 addition(s), 0 copie(s) and 0 deletion(s).
Request b903f204-d92f-41df-8b4e-eca184ad1083: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/preupload/main (authenticated: True)
No LFS files to upload.
Request 4d2a3898-8ad0-4999-8811-5ff1de4bf515: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/main (authenticated: True)


CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/583599d7deac57577b80125c3f686ad9b633fb6e', commit_message='Upload 1.jsonl with huggingface_hub', commit_description='', oid='583599d7deac57577b80125c3f686ad9b633fb6e', pr_url=None, pr_revision=None, pr_num=None)

In [11]:

df.reindex()

,q1,a1,q2,a2,database
0,ある大災害に直面して、人類が存亡の危機に瀕しているゲームを想像してほしい。それを防ぐために、...,主婦は家庭を維持しながら、多くの人たちを支えてきました。彼女たちは食料の確保から、医療に至る...,バンカーでの生活は非常に厳しいと想像されます。主婦は、そんな生活を支えるために何を持って行く...,バンカーでの生活を支えるために、主婦は必要十分な食料や水を確保することが重要です。また、医療...,llm-jp/oasst2-33k-ja
1,なぜニューイングランド・ペイトリオッツはNFL史上最も偉大なチームなのか？,ニューイングランド・ペイトリオッツは、アメリカンフットボールリーグであるNFLの史上、最多の...,その他にもニューイングランド・ペイトリオッツが偉大なチームである理由は何か？,ニューイングランド・ペイトリオッツのコーチとして有名なビル・ベリッチェは、NFL史上最多のス...,kunishou/databricks-dolly-15k-ja
2,フライドポテト、KFC、リンゴ、フルーツサラダ、ステーキ、ビール、ワイン、ウィスキー、ポテト...,リンゴ、フルーツサラダ、ステーキ、レンガ、アロエベラは健康に良い食品です。一方、フライドポテ...,食品以外のものは何でしょうか？,レンチ、砂利、砂は食品ではありません。,kunishou/databricks-dolly-15k-ja
3,BMWとメルセデスの違いは何ですか？,BMWとメルセデスは、両社が独特の特徴を持った高級車の製造に関わる2つの主要企業です。BMW...,それではBMWの代表的なモデルは何ですか？,BMWの代表的なモデルとしては、5系やX5などが挙げられます。これらの車種は、高いパフォーマ...,llm-jp/oasst2-33k-ja
4,インドのハイデラバードについて教えてください。\n\nハイデラバード（/ˈhaɪ dər-ə...,ご説明ありがとうございます。ハイデラバードは面積も生産高も大きく、歴史も長い都市ですね。,それにしても、どのような風景が見られるのでしょうか？,ハイデラバードは丘陵地帯にあり、人工湖もあります。歴史的建造物も多く、一般的な都市とは違った...,kunishou/databricks-dolly-15k-ja
...,...,...,...,...,...
7470,韓国語の発音が難しく感じられて、どう学ばなければいいか気になります。\n\n類似主題：\n日...,韓国語の発音を学ぶには、まずは簡単な言葉や文章を耳に入れることが大切です。音声materia...,日本語の読み方が難しく感じられます。どうしたらよいでしょうか？\n\n类似主题：\n韓国語の...,日本語の読み方を学ぶ際は、まずはひらがなやカタカナの基礎をマスターすることが大切です。その後...,minnnade
7471,以下の手順に従って計算を行なってください。\n1. 数値16を3で掛けた結果から50を引き、...,"1. 16 × 3 = 48, 48 - 50 = -2, -2 + 20 = 18\n2....",以下の手順に従って計算を行なってください。\n1. 数値20を7で掛けた結果から60を引き、...,"1. 20 × 7 = 140, 140 - 60 = 80, 80 + 30 = 110\...",minnnade
7472,同じ父母の両方の子どもがいる場合の顔認証システムの課題と解決策について,同じ父母の両方の子どもは、顔の特徴が似通っていることが多いため、顔認証システムで誤認識する可...,顔認証システムにおけるセキュリティ脅威とその対策について,顔認証システムは、不正アクセスを受けた場合に脆弱性を示す可能性があります。これを防ぐためには...,minnnade
7473,プログラミングにおいてネットワーク通信用ライブラリを利用した開発を行っているのですが、時折障...,ありがとうございます。障害の原因は多岐に渡りますが、考えられる可能性としては、ネットワーク接...,その中でも、ネットワーク接続の不安定が原因だと考えられる場合はどのように対処すべきでしょうか。,ネットワーク接続の問題が原因である場合、まずはインターネット接続やファイアウォール設定を確認...,minnnade
